In [1]:
import numpy as np
import pandas as pd
import random
import math
import sys

In [2]:
# df = pd.read_csv("vote/vote.data", delimiter='\t', header=None) 
# df.head()

In [3]:
# format_outputs(df)

In [4]:
def fold_dataset(dataset, k):

    np.random.shuffle(dataset)
    dataset = np.array_split(dataset, k)

    return dataset


def format_outputs(dataset):

    for instance in dataset:
        if instance[-1] == "d":
            instance[-1] = 1
        else:
            instance[-1] = -1
    return dataset


def separate_attributes(dataset):
#     Separate inputs from outputs and return a dictionary.
    
#     Now it is possible to convert outputs to true integers. 
    
#     Arguments: dataset -- Dataset represented as numpy array of arrays.


    dataset = {
        "input": dataset[:, 0:-1],
        "output": dataset[:, -1].astype(int)
    }
    return dataset

In [5]:
class AdaBoost:
#     Implementation of AdaBoost boosting method.
    
#     AdaBoost combines weak learners to create a strong learning hypothesis.
#     Our weak learners are essentially 1-level decision trees, commonly known as 
#     decision stumps.
#     

    def __init__(self, training_set, testing_set):

        self.training_set = training_set
        self.testing_set = testing_set
         # Number of training instances
        self.m_tr = training_set["input"].shape[0] 
        # Number of input attributes(The same for testing)
        self.n_tr = training_set["input"].shape[1]  
        # Number of testing instances
        self.m_ts = testing_set["input"].shape[0]
        # Weights for training instances
        self.weights = np.divide(np.ones(self.m_tr), self.m_tr)  
        # Collection of chosen weak learners
        self.ensemble = [] 
        # Weight assigned to each weak learner
        self.alpha = []  


    def evaluate_stump(self, stump):
#         Returns the stump error in current weighted training set.
        
#         stump -- Stump to be evaluated.
#         

        predictions = np.zeros(self.m_tr)  # Hypothesis for each instance
        pred_errors = np.ones(self.m_tr)  # 0 if correct, 1 if incorrect
        a = stump["attribute"]  # Attribute index in training set
        # Loop through instances
        for i in range(self.m_tr):
            value = self.training_set["input"][i][a]
            output = self.training_set["output"][i]
            if value == stump["value"]:
                predictions[i] = stump["state"]
            else:
                predictions[i] = stump["state"] * -1
            if predictions[i] == output:
                pred_errors[i] = 0

        # Should divide by the sum of the weights, but it is always 1
        error = np.sum(np.multiply(self.weights, pred_errors))

        return error, predictions


    def find_best_stump(self):
#         Return the best decision stump for current weights.

#         Creates different decision stumps( attributes * 3 possible values
#         * 2 states[True or False] combinations). 


        best_stump = {}
        lowest_error = float("inf")
        possible_values = ["y", "n", "?"]
        possible_states = [1, -1]
        # Loop through attributes
        for a in range(self.n_tr):
            for value in possible_values:
                for state in possible_states:
                    # Instantiates stump
                    stump = {"attribute": a}
                    stump["value"] = value
                    # Predict this for value or -1 * this for not value
                    stump["state"] = state  

                    # Calculate error for stump
                    error, predictions = self.evaluate_stump(stump)
                    stump["error"] = error
                    stump["predictions"] = predictions

                    if error < lowest_error:
                        lowest_error = error
                        best_stump = stump

        return best_stump


    def calculate_alpha(self, model):
#         Calculates alpha for the error of the given(best) model.
        
#         Attributes: model = Best predicting weak learner for time t.
#       
        error = model["error"]
        alpha = 0.5 * np.log((1 - error) / error)
        
        return alpha


    def update_weights(self, model, alpha):
#         Update weights for time t according to AdaBoost's formula.

        self.weights = np.multiply(self.weights, 
                                   np.exp(-1 * alpha 
                                     * np.multiply(self.training_set["output"],
                                                   model["predictions"])
                                   )
                       )
        
        self.weights = np.divide(self.weights, np.sum(self.weights))


    def evaluate_ensemble(self):
        #Evaluate current strong learner with the testing set

        correct = 0
        # Loop through instances
        for i in range(self.m_ts):
            H = 0  
            for model in range(len(self.ensemble)):
                # Get the attribute that the model is related with
                a = self.ensemble[model]["attribute"]
                # Get the value(class) it presents in this instance
                value = self.testing_set["input"][i][a]
                # Predict according to model rules
                if value == self.ensemble[model]["value"]:
                    prediction = self.ensemble[model]["state"]
                else:
                    prediction = self.ensemble[model]["state"] * -1
                H += self.alpha[model] * prediction
            H = np.sign(H)  # Strong model hypothesis

            if H == self.testing_set["output"][i]:
                correct += 1

        accuracy = (correct / self.m_ts) * 100  # Simple accuracy measure
        error = 100 - accuracy

        return accuracy, error


    def boost(self, num_iterations):

        accuracies = []  # Accuracy per iteration
        errors = []  # Error per iteration
        model_errors = []  # Errors for the best model in each iteration
        for i in range(num_iterations):
            best_model = self.find_best_stump()
            model_errors.append(best_model["error"] * 100)
            self.ensemble.append(best_model)
            self.alpha.append(self.calculate_alpha(best_model)) 

            results = self.evaluate_ensemble()
            accuracies.append(results[0])
            errors.append(results[1])

            self.update_weights(best_model, self.alpha[i])
        return accuracies, errors, model_errors


In [6]:
cv_accuracies = []
cv_errors = []
cv_model_errors = []

In [7]:

k = 10  # Number of folds

# Dataset retrieving and formatting
dataset = pd.read_csv("vote/vote.data", delimiter='\t', header=None) 
dataset = np.array(dataset)
dataset = format_outputs(dataset)
dataset = fold_dataset(dataset, k)


# Execute k-fold cross-validation
for i in range(k):
    testing_set = separate_attributes(dataset[i])
    remaining_folds = np.concatenate(np.delete(dataset, i))
    training_set = separate_attributes(remaining_folds)

#     ada = AdaBoost(training_set, testing_set)
#     results = ada.boost(301)

#     cv_accuracies.append(results[0])
#     cv_errors.append(results[1])
#     cv_model_errors.append(results[2])

# # Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

# # Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)



In [8]:
# print(cv_accuracies[300])
# print(cv_errors[300])

In [9]:
training_set['input'].shape

(392, 16)

In [10]:
training_set

{'input': array([['n', 'n', 'n', ..., 'y', 'n', 'y'],
        ['y', 'n', 'y', ..., 'n', 'y', '?'],
        ['y', 'y', 'y', ..., 'n', 'y', 'y'],
        ...,
        ['y', 'y', 'y', ..., 'n', 'y', '?'],
        ['n', 'y', 'n', ..., 'y', 'n', 'n'],
        ['y', 'y', 'y', ..., 'y', 'n', 'y']], dtype=object),
 'output': array([-1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1,  1,
        -1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1, -1,  1,
         1,  1,  1,  1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
         1,  1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1,
         1,  1,  1, -1, -1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1, -1,  1,
        -1,  1, -1,  1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1, -1,
         1,  1,  1, -1, -1, -1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
         1, -1,  1,  1,  1, -1,  1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1,
        -1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,  1,  1

In [11]:
training_set_input_1 = training_set['input'][:19] #5%
training_set_output_1 = training_set['output'][:19]
training_set1 = {'input': training_set_input_1, 'output': training_set_output_1}


In [12]:
training_set_input_2 = training_set['input'][:58] #15%
training_set_output_2 = training_set['output'][:58]
training_set2 = {'input': training_set_input_2, 'output': training_set_output_2}

training_set_input_3 = training_set['input'][:117] #30%
training_set_output_3 = training_set['output'][:117]
training_set3 = {'input': training_set_input_3, 'output': training_set_output_3}

training_set_input_4 = training_set['input'][:196] #50%
training_set_output_4 = training_set['output'][:196]
training_set4 = {'input': training_set_input_4, 'output': training_set_output_4}

training_set_input_5 = training_set['input'][:274] #70%
training_set_output_5 = training_set['output'][:274]
training_set5 = {'input': training_set_input_5, 'output': training_set_output_5}

training_set_input_6 = training_set['input'][:293] #100%
training_set_output_6 = training_set['output'][:293]
training_set6 = {'input': training_set_input_6, 'output': training_set_output_6}


In [13]:
ada1 = AdaBoost(training_set1, testing_set)
results = ada1.boost(301)

cv_accuracies.append(results[0])
cv_errors.append(results[1])
cv_model_errors.append(results[2])

## Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

## Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)

In [20]:
print(cv_accuracies[0][300])
print(cv_errors[0][300])

93.02325581395348

In [21]:
cv_accuracies = []
cv_errors = []
cv_model_errors = []
ada2 = AdaBoost(training_set2, testing_set)

results = ada2.boost(301)
cv_accuracies.append(results[0])
cv_errors.append(results[1])
cv_model_errors.append(results[2])

# # Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

# Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)

In [22]:
print(cv_accuracies[0][300])
print(cv_errors[0][300])

93.02325581395348
6.976744186046517


In [23]:
cv_accuracies = []
cv_errors = []
cv_model_errors = []
ada3 = AdaBoost(training_set3, testing_set)

results = ada3.boost(301)
cv_accuracies.append(results[0])
cv_errors.append(results[1])
cv_model_errors.append(results[2])

# # Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

# Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)

In [24]:
print(cv_accuracies[0][300])
print(cv_errors[0][300])

88.37209302325581
11.627906976744185


In [25]:
cv_accuracies = []
cv_errors = []
cv_model_errors = []
ada4 = AdaBoost(training_set4, testing_set)

results = ada4.boost(301)
cv_accuracies.append(results[0])
cv_errors.append(results[1])
cv_model_errors.append(results[2])

# # Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

# Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)

In [26]:
print(cv_accuracies[0][300])
print(cv_errors[0][300])

90.69767441860465
9.302325581395351


In [27]:
cv_accuracies = []
cv_errors = []
cv_model_errors = []
ada5 = AdaBoost(training_set5, testing_set)

results = ada5.boost(301)
cv_accuracies.append(results[0])
cv_errors.append(results[1])
cv_model_errors.append(results[2])

# # Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

# Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)

In [28]:
print(cv_accuracies[0][300])
print(cv_errors[0][300])

90.69767441860465
9.302325581395351


In [29]:
cv_accuracies = []
cv_errors = []
cv_model_errors = []
ada6 = AdaBoost(training_set6, testing_set)

results = ada6.boost(301)
cv_accuracies.append(results[0])
cv_errors.append(results[1])
cv_model_errors.append(results[2])

# # Convert lists to numpy arrays for faster calculations
# cv_accuracies = np.asarray(cv_accuracies)
# cv_errors = np.asarray(cv_errors)
# cv_model_errors = np.asarray(cv_model_errors)

# Calculate the mean of the accuracies and the errors
# cv_accuracies = np.divide(np.sum(cv_accuracies, axis=0), k)
# cv_errors = np.divide(np.sum(cv_errors, axis=0), k)
# cv_model_errors = np.divide(np.sum(cv_model_errors, axis=0), k)

In [30]:
print(cv_accuracies[0][300])
print(cv_errors[0][300])

93.02325581395348
6.976744186046517
